POWERMAX_Tonka
Notebook created 03/30/23 from OptionGun03-Tonka-Prdtn
Currently includes ONLY OptionGun03-Bullets
-Fixed crash on bad option chains.
-Added many tickers.
-Added Bullets_PSTO in OPtionGunLib01.py. NOte this is not a Jupyuter notebook!

User Notes
1. This program needs a user key from Darqube.com, which it uses to get real-time stock prices.
   Go to the Darqube website get a user account and choose the free service. Then get your key.
   Find the Dar_key line just below the "User Parameters" block in the cell below.
   Enter it between the quotes on the line:  DAR_key = ''
   
2. OptionGun produces these csv files:
     root_data.csv. This contains a list of all tickers, with real-time price at the time of execution, 
     and the netx Earnings Report date for each ticker, if available. DO NOT CHANGE THIS FILE.
     all_optinos_suffix.csv. This file contains every option available for every ticker in the list. 
     Put_STO_short_Tonka.csv. Best STO options that meet criteria and expiry < 50 days out.
     Put_STO_mid_Tonka.csv.   Best STO options that meet criteria and 50 days <= expiry < 150 days out.
     Put_STO_long_Tonka.csv   Best STO options that meet criteria and 150 days out <= expiry.
     
     Call_STO_short_Tonka.csv. Best STO options that meet criteria and expiry < 50 days out.
     Call_STO_mid_Tonka.csv.   Best STO options that meet criteria and 50 days <= expiry < 150 days out.
     Call_STO_long_Tonka.csv   Best STO options that meet criteria and 150 days out <= expiry.

     
2. The parameters just under the "User Parameters" header block can be changed.
    Add and remove ticker symbols from the TICKERS = list. Enclose each ticker Follow the format shown.
    HIDE_TICKERS. You may want to collect option info for a ticker, but not see that ticker in best_file.csv.
    Just put the ticker to be hidden in this list.
    FILE_ID_TAG is the suffix that OptionGun will attach to all_options and best_options. You can change it at any time.
    
3. PowerMax needs about 2-8 seconds to get the option string for each ticker. For 200 tickers, it will take about 2-25 minutes.

PowerMax Structure

- HEADERS and CONSTANTS

- GET OPTION DATA
       - Get real-time price data from Darqube.
       - Get option strings from yfinance.
       - Get ER dates from yfinance.
       - Parse option strings into common POWMax format.

Future
- Bullets_CSTO- Screen and select CSTO offers
- Bullets_PBTC- Screen and select PBTC offers
- Bullets_CBTC- Screen and select CBTC offers

TODO 04/01/23
-Add TradeManager
-Add ScoreKeeper
-Add CrystalBall

In [1]:
# ****************
# USER ADJUSTABLE PARAMETERS
# *****************

DAR_key = '90180f15ecc74513a01ca017eca2bb4f'

TICKERS = ('BSM', 'BGSF', 'CPA', 'JEF', 'STLD', 'AMR', 'UNVR', 'CVCO', 'HUBG', 'ACLS', 'ASC', 'ASRT', \
           'AFYA', 'SMCI', 'HDSN', 'VIST', 'VRNA', 'NWG', 'PINE', 'VIR', 'ASO', 'PERI', 'STRL', 'VPG', \
           'AAPL', 'ABBV', 'ACRE', 'AFL', 'ADBE', 'AIG', 'AMD', 'AMP', 'AMT', 'AMZN', 'AOS', 'ARCC', 'ARNC', 'AVB', \
           'AVGO', 'AZO', 'BA', 'BAC', 'BBBY', 'BIIB', 'BNTX', 'BRSP', 'CAG', 'CAT', 'CCI', 'CEG', 'CF', 'CFR', 'CL', \
           'CLX', 'CMCSA', 'CMG', 'CPT', 'CRM', 'CRWD', 'CSCO', 'CSX', 'CUBE', 'CVS', 'CVX', 'DAC', 'DDOG', 'DHI', \
           'DLR', 'DOCU', 'DUK', 'DVN', 'EPAM', 'EPD', 'EPR', 'ET', 'EXC', 'EXPE', 'EXR', 'FAF', 'FANG', 'FCNCA', 'FDX', 'FITB', 'FLNG', \
           'FMS', 'FNF', 'FSLY', 'FSR', 'FYBR', 'GBX', 'GD', 'GIS', 'GLNG', 'GOOGL', 'GS', 'GSL', 'HD', 'HIW', \
           'HON', 'HR', 'HRB', 'IBM', 'INTC', 'IPI', 'IRM', 'JEF', 'JNJ', 'JPM', 'JXN', 'K', 'KO', 'LADR', 'LLY', 'LMT', 'LOW', \
           'LRCX', 'LYB', 'MA', 'MAN', 'MCK', 'META', 'MGA', 'MLM', 'MMM', 'MMP', 'MNSO', 'MOD', 'MOS', 'MPW', \
           'MRK', 'MRNA', 'MRVL', 'MS', 'MSFT', 'MTB', 'MTCH', 'MU', 'NFLX', 'NKE', 'NKTR', 'NMM', 'NOW', 'NRG', 'NUE', 'NVDA', \
           'OKTA', 'ON', 'ORCC', 'PACW', 'PANW', 'PB', 'PD', 'PFE', 'PFG', 'PGR', 'PHM', 'PII', 'PLTR', 'PNC', 'PRU', 'PSA', 'PYPL', 'QCOM', \
           'RC', 'RITM', 'RIVN', 'ROST', 'RTX', 'SB', 'SBLK', 'SBUX', 'SCHW', 'SEDG', 'SKT', 'SNOW', 'SO', 'SPG', 'SPY', 'SRC', \
           'STAG', 'STWD', 'TMUS', 'TSLA', 'TRV', 'TXN', 'TSCO', 'TSN', 'UBS', 'UHAL', 'UNP', 'UPS', 'URI', 'USAC', 'USB', 'V', \
           'VICI', 'VGT', 'VLO', 'VMW', 'VNOM', 'VOO', 'WDC', 'WEC', 'WMT', 'WSM', 'WFC', 'XOM', 'ZIM', 'ZS')

HIDE_TICKERS = ('AOS', 'FSLY', 'NOW', 'PLTR', 'TSLA', 'VNOM', 'RIVN', 'NKTR') # Keep these tickers for database, but do not show them in best_options, because you do not want to own them.  




In [2]:
# ****************
# OptionGun03-Bullets - 03/30/23
# *****************
# Goal - OptionGun downloads option strings from yahoo yfinance. It generates option metris
#        and screens the options that score the best on these metrics: POW, ARR, PctOTM, and Expiry DaysOut.
#
# 
#   ToDo - Move GetERDates to CrystalBall. Not part of OptionGun.
#          Delete extra columns from saved STO profiles. What went wrong with existing delete? Reconcile puts_all with Putsssa
#
#   03/26/23 - Got Bullets_STO working. Remove extra cols, check results.
#   03/19/23 - Moved 3 fcts into OptionGunLib.
#   03/18/23 - Trapped tickers with no data to stop execution crash. Now it captures the ticker and keeps running.
#   12/16/22 - Removed 'inTheMoney' column since it seems to cause 'mixed values' errors when program runs.
#   11/27/22 - Remove IEX. Now using Darqube for market data. Added many more tickers, approx. 150 total.
#   02/19/22 - Bought IEX subscription. Started trading options again. Woohoo!
#   10/13/19 - Rewrite all_options section to eliminate all selection criteria. all_options ONLY gets option data from yahoo.
#
# *****************
#
# Import Libraries
#
# *****************

import csv
import copy
import json
import numpy as np
import os
import pandas as pd
import requests   # for http requests
import scipy
from   scipy import stats
from   scipy.stats import norm
import time
import yfinance as yf

import datetime
from   datetime import datetime
from   datetime import date
from   datetime import timedelta
#from   datetime import fromtimestamp

from   dateutil.relativedelta  import relativedelta
from   yahoo_earnings_calendar import YahooEarningsCalendar

import OptionGunLib01
from OptionGunLib01 import GetERDates
from OptionGunLib01 import GetPrices
from OptionGunLib01 import GetOptions
from OptionGunLib01 import BuildOptionMetrics
from OptionGunLib01 import Bullets_PSTO

import TradeManagerLib01
#from TradeManagerLib01 import BrokerReporter

# *****************
#
# Flow Control
#
# CHECK 3 PARAMS:  TICKERS, FILE_ID_TAG, profile
#
# *****************

update_root_data       = True  #skip Darqube call for tests. 
update_root_data_ER    = False  #skip call to yahoo calendar for tests. 
update_option_data     = True  #skip call to yfinance if testing code.
run_BuildOptionMetrics = update_option_data  #skip call to BuildOptionMetrics if no update of option data.

# *****************
#
# CONSTANTS and PARAMETERS
#
# *****************

TICKERS = ('PB', 'AMZN', 'FRC', 'NFLX') # Short ticker list for testing and targeted trade info..
print(len(TICKERS), ' tickers to analyze.')
FILE_ID_TAG       = '_Tonka'

DATE_FORMAT   = '%Y-%m-%d %H:%M:%S'

TICKER_DELAY  = 0.015  # seconds between tickers for data requests.
EXPIRY_DELAY  = 0.025
YEC_DELAY     = 0.75

MAX_DAYSOUT   = 600.0  # This constant controls how many expiry strings are downloaded. Use low # to exclude BTOS and reduce ext. calls. Hi # includes BTOS.
MIN_DAYSOUT   = -1.0   

FEE_SPREAD    = 0.75     # Fee in the bid/ask range.
DAYS_TO_CHECK_ER = (1,4) #Update ER dates from yahoo on these days of the month.

call_ticker_options = pd.DataFrame()
put_ticker_options  = pd.DataFrame()
ticker_options      = pd.DataFrame() # for a single ticker
all_options         = pd.DataFrame() # includes options for all tickers. Do not use any limits or exclusions for all_options.
good_options        = pd.DataFrame() # Has rankings for CSTO options and BTOs (CBTO).
stats               = pd.DataFrame() # Not used now.
root_data           = pd.DataFrame() # persistent file used by other programs. Specifically, OptionJrnlAll spreadsheet. This programu updates it.
options_open        = pd.DataFrame() # persistent file used by OptionCloser to recommend BTC offers.


OptionGunLib01 runs like a top.
TradeManagerLib01 is empty now.
4  tickers to analyze.


In [3]:
# *****************
#
# MAIN PROGAM STARTS HERE
#
# 1. Call GetPrices to get real-time price data from Darqube write to root_data. 
# 2. Call GetERDates to get dates from Yahoo Earnings Calendar and write to root_data.
# 3. Call GetOptions to get options data from yfinance. Use "try" statements to skip over exp dates where call to yfinance failed.
# 4. Call BuildOptionMetrics to build additional metrics fields in each option record.
# 5. Call Bullets_STO to screen all_options for each STO profile.
# 6. Call other Bullets fcts as they get written.
# 
# *****************

root_data = pd.read_csv('root_data.csv')   #Get root_data price and ER info.
root_data.set_index('Ticker', inplace = True, drop = True)
print('just read root_data')
#Use Darqube for stock price data - both market and historical.

print('start Darqube access at:  ', datetime.now(), '  Done in ',len(TICKERS) * 1.0 / 200.0, ' minutes.')
if update_root_data:
    root_data = GetPrices(TICKERS, root_data, DAR_key)
print(datetime.now())

if update_root_data_ER:                # skip call to YEC for ER update
    if date.today().day in DAYS_TO_CHECK_ER:
        print('Updating ER dates.')
        root_data = GetERDates(root_data, TICKERS, YEC_DELAY)
root_data.to_csv('root_data.csv')

# Get option chains
if update_option_data:
    all_options = GetOptions(root_data, TICKERS, EXPIRY_DELAY, TICKER_DELAY, MAX_DAYSOUT, MIN_DAYSOUT)
else:
    all_options = pd.read_csv('all_options23-03-29-12.csv', index_col = 0) # a test file
    
#Calculate additional columns for all_options.
#Prep clean_options for use by Bullets.
if run_BuildOptionMetrics:
    all_options, clean_options = BuildOptionMetrics(all_options, FEE_SPREAD, HIDE_TICKERS, DATE_FORMAT)
    write_file = 'all_options' + FILE_ID_TAG + '.csv'
    all_options.to_csv(write_file)
    print('Wrote all_options')
else:
    all_options = pd.read_csv('all_options23-03-24-12.csv') # For testing only.
    
#
# Prep option_profiles for use in Bullets
profiles = pd.read_csv('option_profiles.csv', index_col = 0)
float_columns = profiles.columns.tolist()
float_columns.remove('Active') # So, why have 'Active'? Maybe for later testing and run control?.
profiles[float_columns] = profiles[float_columns].astype(float).copy()

print('calling Bullets_PSTO')
Bullets_PSTO(clean_options, profiles)

print('Bullets_PSTO ran all the way.')


just read root_data
start Darqube access at:   2023-04-05 21:51:41.562057   Done in  0.02  minutes.
2023-04-05 21:51:44.596972
getting data for  PB  at time  2023-04-05 21:51:44.596972
getting data for  AMZN  at time  2023-04-05 21:51:47.597151
getting data for  FRC  at time  2023-04-05 21:52:00.764027
getting data for  NFLX  at time  2023-04-05 21:52:09.480346
Got all the data for all tickers. Created additional cols. time =  2023-04-05 21:52:19.233810
     Got options for every ticker.
Wrote all_options
calling Bullets_PSTO


KeyError: "['Select'] not found in axis"